In [1]:
import sys
import os
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)
import kinodyn_um.urdfparser as u2c
import numpy as np
import casadi as ca

In [2]:
alpha = u2c.URDFparser()
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
path_to_urdf = os.path.join(
    project_root,
    'usage',
    'urdf',
    'alpha_5_robot_for_FD.urdf'
)
alpha.from_file(path_to_urdf)

root = "base_link"
tip = "alpha_standard_jaws_base_link"
gravity = [0, 0, -9.81]
n_joints = alpha.get_n_joints(root, tip)

qddot_sym = alpha.get_forward_dynamics_crba(root, tip)
# ─── 2. Define CasADi symbols and build the ODE ─────────────────────────────
q     = ca.MX.sym('q',     n_joints)
q_dot = ca.MX.sym('q_dot', n_joints)
tau   = ca.MX.sym('tau',   n_joints)
dt    = ca.MX.sym('dt')
gravity    = ca.MX.sym('gravity')

viscous = ca.MX.sym('viscous', n_joints)
coulomb = ca.MX.sym('coulomb', n_joints)
I_Grotor  = ca.MX.sym("Jm",    n_joints)             # rotor inertias
payload  = ca.MX.sym("payload")


# # hard limits → CasADi DM (size = n_joints)
# q_min = ca.DM([1.00, 0.01, 0.01, 0.01])
# q_max = ca.DM([5.50, 3.40, 3.40, 5.70])

# # 1. hard clamps used *only* in dynamics ------------------------------
# q_safe = ca.fmin(ca.fmax(q, q_min), q_max)

# qdot_limited = ca.if_else(q_safe <= q_min,  ca.fmax(q_dot, 0),  q_dot)
# q_dot_safe   = ca.if_else(q_safe >= q_max,  ca.fmin(qdot_limited, 0), qdot_limited)

q_ddot = qddot_sym(q, q_dot, tau,
                   gravity, viscous, coulomb, I_Grotor, payload)

# 2. state & RHS -------------------------------------------------------
x     = ca.vertcat(q, q_dot)                   # ← pure symbols
x_dot = ca.vertcat(q_dot, q_ddot) * dt    # RHS may be complex
p = ca.vertcat(dt,gravity,viscous, coulomb, I_Grotor, payload)
# 3. integrator --------------------------------------------------------
dae  = {'x': x, 'ode': x_dot, 'p': p, 'u': tau}
opts = {'simplify': True,
        'number_of_finite_elements': 200}      # default grid [0,1]

intg = ca.integrator('intg', 'rk', dae, 0, 1, opts)  # ◀ no extra 0,1

# next-state map
res    = intg(x0=x, u=tau, p=p)
x_next = res['xf']
# F_next = ca.Function('F_next', [x, tau, p], [x_next])

F_next_ = ca.Function('Mnext', [x, tau, dt, gravity,viscous, coulomb, I_Grotor, payload], [x_next])

In [3]:
# c , cpp or matlab code generation for forward dynamics
F_next_.generate("F_next_.c")
os.system(f"gcc -fPIC -shared F_next_.c -o libMnext.so")

0